In [1]:
!pip3 install numpy pillow torch torchvision

In [18]:
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms 

In [19]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [4]:
train_data = torchvision.datasets.CIFAR10(root="./data", train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root="./data", train=False, transform=transform, download=True)


train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

In [5]:
image , label = train_data[0]

In [6]:
image.size()

torch.Size([3, 32, 32])

In [7]:
class_name = ["plane","car","bird","cat","deer","dog","frog","horse","ship","truck"]

In [8]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()

        self.conv1 = nn.Conv2d(3,12,5) #(12,28,28)
        self.pool = nn.MaxPool2d(2,2) #(12,14,14)
        self.conv2 = nn.Conv2d(12,24,5) #(24,10,10)
        self.fc1 = nn.Linear(24 * 5 * 5,120) 
        self.fc2 = nn.Linear(120,84) 
        self.fc3 = nn.Linear(84,10) 

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x 

In [9]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [10]:
for epoch in range(35):
    print(f"training epoch {epoch}...")

    running_loss = 0.0

    for i , data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs,labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"loss: {running_loss/len(train_loader): .4f}")

training epoch 0...
loss:  2.2473
training epoch 1...
loss:  1.7827
training epoch 2...
loss:  1.5449
training epoch 3...
loss:  1.4130
training epoch 4...
loss:  1.3120
training epoch 5...
loss:  1.2274
training epoch 6...
loss:  1.1618
training epoch 7...
loss:  1.1028
training epoch 8...
loss:  1.0498
training epoch 9...
loss:  1.0036
training epoch 10...
loss:  0.9662
training epoch 11...
loss:  0.9252
training epoch 12...
loss:  0.8906
training epoch 13...
loss:  0.8581
training epoch 14...
loss:  0.8236
training epoch 15...
loss:  0.7906
training epoch 16...
loss:  0.7649
training epoch 17...
loss:  0.7379
training epoch 18...
loss:  0.7067
training epoch 19...
loss:  0.6856
training epoch 20...
loss:  0.6580
training epoch 21...
loss:  0.6333
training epoch 22...
loss:  0.6064
training epoch 23...
loss:  0.5868
training epoch 24...
loss:  0.5628
training epoch 25...
loss:  0.5450
training epoch 26...
loss:  0.5197
training epoch 27...
loss:  0.4980
training epoch 28...
loss:  0.

In [11]:
torch.save(net.state_dict(), "trained_net.pth")

In [12]:
import os
print(os.listdir())

['.anaconda', '.conda', '.condarc', '.continuum', '.dotnet', '.idlerc', '.ipynb_checkpoints', '.ipython', '.jupyter', '.matplotlib', '.node_repl_history', '.python_history', '.redhat', '.spyder-py3', '.templateengine', '.thumbnails', '.virtual_documents', '.vscode', '.wslconfig', 'anaconda_projects', 'ansel', 'app.js', 'AppData', 'appimg.js', 'Application Data', 'bot.js', 'bot.json', 'BrawlhallaReplays', 'Contacts', 'Cookies', 'curseforge', 'data', 'Documents', 'Downloads', 'EDA', 'Favorites', 'fileread.c', 'fileread.exe', 'gcc', 'get-pip.py', 'imageclassificationsystem.ipynb', 'Input.txt', 'Jedi', 'Links', 'Local Settings', 'Music', 'My Documents', 'NetHood', 'node_modules', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{b8752c82-cf55-11ef-96b8-b36d0caa1413}.TM.blf', 'NTUSER.DAT{b8752c82-cf55-11ef-96b8-b36d0caa1413}.TMContainer00000000000000000001.regtrans-ms', 'NTUSER.DAT{b8752c82-cf55-11ef-96b8-b36d0caa1413}.TMContainer00000000000000000002.regtrans-ms', 'ntuser.ini'

In [13]:

torch.save(net.state_dict(), "trained_net_.pth")

In [14]:
net = NeuralNet()
net.load_state_dict(torch.load("trained_net_.pth"))
net.eval()

NeuralNet(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=600, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [15]:
correct = 0 
total = 0 
net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs,1)
        total += labels.size(0)
        correct += (predicted  == labels).sum().item()

accuracy = 100 * correct / total
print(f"accuracy: {accuracy}%")


accuracy: 68.37%


In [21]:
new_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

def load_image(image_path):
    image = Image.open(image_path)

    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

image_paths = ["Downloads/car.jpeg","Downloads/cat.jpeg","Downloads/dog.jpeg","Downloads/plane.jpg"]

images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f"prediction: {class_name[predicted.item()]}")

prediction: car
prediction: cat
prediction: dog
prediction: plane
